## Airtable

**Airtable** es una plataforma online fácil de usar para crear y compartir bases de datos. La interfaz es sencilla, amigable y permite a cualquier persona crear una base de datos.

_**Documentación:** https://airtable.com/developers/web/api/introduction_

In [ ]:
import numpy as np
import pandas as pd

import requests

from pprint import pprint

In [ ]:
# Versiones

print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"requests=={requests.__version__}")

#### Authentication - Tokens 

In [ ]:
TOKEN = "patJ9tx1cx5VY0mYl.abcdefgh1234567890ABCDEFGHIJKLMNOPQRSTUVWXY1234567890" # Usuario

BASE_ID = "app2hLPOGc7G1UjfX" # Base: Ensaladas - API

TABLE_ID = "tbl42rhIPJQkrMdbH" # Tabla: datos_1

airtable_base_url = "https://api.airtable.com/v0"

In [ ]:
# Headers
headers = {"Authorization" : f"Bearer {TOKEN}",
           "Content-Type"  : "application/json"}

pprint(headers)

### 01. List Records

```html
endpoint: https://api.airtable.com/v0/{BASE_ID}/{TABLE_ID}
HTTP method: GET
```

**`Params:`**
- **fields** : _array of strings_ : `optional`
    - Only data for fields whose names are in this list will be included in the result. If you don't need every field, you can use this parameter to reduce the amount of data transferred.


- **filterByFormula** : _string_ : `optional`
    - A [formula](https://support.airtable.com/docs/formula-field-reference) used to filter records. The formula will be evaluated for each record, and if the result is not 0, false, "", NaN, [], or #Error! the record will be included in the response. We recommend testing your formula in the Formula field UI before using it in your API request. If combined with the view parameter, only records in that view which satisfy the formula will be returned. The formula must be encoded first before passing it as a value. You can use this tool to not only encode the formula but also create the entire url you need.


- **maxRecords** : _number_ : `optional`
    - The maximum total number of records that will be returned in your requests. If this value is larger than pageSize (which is 100 by default), you may have to load multiple pages to reach this total.
    
    
- **pageSize** : _number_ : `optional`
    - The number of records returned in each request. Must be less than or equal to 100. Default is 100.
    
    
- **sort** : _array of objects_ : `optional`
    - A list of sort objects that specifies how the records will be ordered. Each sort object must have a field key specifying the name of the field to sort on, and an optional direction key that is either "asc" or "desc". The default direction is "asc". The sort parameter overrides the sorting of the view specified in the view parameter. If neither the sort nor the view parameter is included, the order of records is arbitrary.
    
    
        - For example, to sort records by Name in descending order, send these two query parameters:
            - sort%5B0%5D%5Bfield%5D=Name
            - sort%5B0%5D%5Bdirection%5D=desc
            
          For example, to sort records by Name in descending order, pass in:
            - [{field: "Name", direction: "desc"}]
            
- **cellFormat** : _string_ : `optional`
    - The format that should be used for cell values. Supported values are:
        - **json**: cells will be formatted as JSON, depending on the field type.
        - **string**: cells will be formatted as user-facing strings, regardless of the field type. The **timeZone** and **userLocale** parameters are required when using string as the **cellFormat**.
        - The default is **json**.


- **timeZone** : _string_ : `optional`
    - The time zone that should be used to format dates when using string as the **cellFormat**. This parameter is required when using string as the **cellFormat**.
    

- **userLocale** : _string_ : `optional`
    - The user locale that should be used to format dates when using string as the **cellFormat**. This parameter is required when using string as the **cellFormat**.
    
    
- **returnFieldsByFieldId** : _boolean_ : `optional`
    - An optional boolean value that lets you return field objects where the key is the field id.
    - his defaults to **false**, which returns field objects where the key is the field name.

In [ ]:
# List Records

# Endpoint
endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

params = {"fields"                : None, 
          "maxRecords"            : None, 
          "pageSize"              : None,
          "returnFieldsByFieldId" : None}

print(endpoint)

pprint(params, sort_dicts = False)

In [ ]:
response = requests.get(url = endpoint, headers = headers, params = params)

print(f"response: {response.status_code}")

print(f"endpoint: {response.url}")

print("-"*120)

pprint(response.json(), sort_dicts = False)

print("-"*120)

records_id = [x["id"] for x in response.json()["records"]]

In [ ]:
print(len(response.json()["records"]))

**Notas de la documentación**:

```html
Pagination
The server returns one page of records at a time. Each page will contain pageSize records, which is 100 by default.

If there are more records, the response will contain an offset. To fetch the next page of records, include offset in the next request's parameters.

Pagination will stop when you've reached the end of your table. If the maxRecords parameter is passed, pagination will stop once you've reached this maximum.
```

Si quisieramos extraer más de 100 elementos, debemos usar el parámetro **offset** en el endpoint **GET**. La primera llamada no tendrá el parámetro **offset** pero retornará en el **JSON**  el **offset** que debemos agregar en la siguiente llamada como paràmetro.

In [ ]:
%%time

params = {}

endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

datos = list()

while params.get("offset") != None or len(datos) == 0:
    
    response = requests.get(url = endpoint, headers = headers, params = params)
    
    print(f"response: {response.status_code}")
    print(f"endpoint: {response.url}")
    
    data = response.json()
    
    offset = data.get("offset")
    
    params["offset"] = offset
    
    datos.extend(data["records"])
    
print(len(datos))

### 02. Retrieve a Record

```html
endpoint: https://api.airtable.com/v0/{BASE_ID}/{TABLE_ID}/{RECORD_ID}
HTTP method: GET
```

In [ ]:
# Retrieve a Record

# Endpoint

RECORD_ID = records_id[0]
endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}/{RECORD_ID}"

print(endpoint)

In [ ]:
response = requests.get(url = endpoint, headers = headers)

print(f"response: {response.status_code}")

print(f"endpoint: {response.url}")

print("-"*120)

pprint(response.json(), sort_dicts = False)

print("-"*120)

### 03. Create Records

```html
endpoint: https://api.airtable.com/v0/{BASE_ID}/{TABLE_ID}
HTTP method: POST
```

**Notas de la documentación**:
```html
Your request body should include an array of up to 10 record objects. Each of these objects should have one key whose value is an inner object containing your record's cell values, keyed by either field name or field id.

Returns an array of record objects created if the call succeeded, including record IDs which will uniquely identify the records within Tabla API.
```

**Estructura de los datos**:
```python
{"records" : [{"fields" : {}},
              {"fields" : {}}],
 "typecast" : True}
```

- _"typecast" : True_ permite a Airtable castear los elementos al tipo de dato correcto para cada columna en caso de ser posible.
- Airtable no permite valores NaN's, para evitar errores debemos llenar los NaN's con el valor **None**.

In [ ]:
# Create Records

# Endpoint
endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

print(endpoint)

In [ ]:
df = pd.read_csv("todas_recetas.csv")

df = df.sample(10).replace({np.nan : None})

df

In [ ]:
# Primera Fila

pprint(df.iloc[0, :].to_dict())

In [ ]:
# Bucle normal

records = {"records"  : [],
           "typecast" : True}

for i in range(df.shape[0]):
    
    fila = {"fields" : df.iloc[i, :].to_dict()}
    
    records["records"].append(fila)
    
pprint(records)

In [ ]:
# List Comprehensions

pprint({"records" : [{"fields" : df.iloc[i, :].to_dict()} for i in range(df.shape[0])],
        "typecast" : True})

datos_subir = {"records" : [{"fields" : df.iloc[i, :].to_dict()} for i in range(df.shape[0])],
               "typecast" : True}

In [ ]:
response = requests.post(url = endpoint, json = datos_subir, headers = headers)

print(f"response: {response.status_code}")

print(f"endpoint: {response.url}")

print("-"*120)

pprint(response.json(), sort_dicts = False)

print("-"*120)

### 04. Update Records

```html
endpoint: https://api.airtable.com/v0/{BASE_ID}/{TABLE_ID}
HTTP method: PATCH
```

**Notas de la documentación**:
```html
Your request body should include an array of up to 10 record objects. Each of these objects should have an id property representing the record ID and a fields property which contains all of your record's cell values by field name or field id for all of your record's cell values by field name.
```

**Estructura de los datos**:
```python
{"records" : [{"id"     : "",
               "fields" : {"field_1" : ""
                           "field_2" : ""}}]}
```

In [ ]:
# Update Records

# Endpoint
endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

print(endpoint)

In [ ]:
datos_modificar = {"records" : [{"id" : x, "fields" : {}} for x in records_id[:10]]}
pprint(datos_modificar)

In [ ]:
for i in range(len(datos_modificar["records"])):
    
    for col in ["nombres", "descripciones", "categorias"]:
        
        datos_modificar["records"][i]["fields"][col] = "HOLA ESTAMOS CAMBIANDO LOS DATOS DE ESTA RECETA"
    
pprint(datos_modificar)

In [ ]:
response = requests.patch(url = endpoint, json = datos_modificar, headers = headers)

print(f"response: {response.status_code}")

print(f"endpoint: {response.url}")

print("-"*120)

pprint(response.json(), sort_dicts = False)

print("-"*120)

### 05. Delete Records

```html
endpoint: https://api.airtable.com/v0/{BASE_ID}/{TABLE_ID}
HTTP method: DELETE
```

**Notas de la documentación**:
```html
Your request should include a URL-encoded array of up to 10 record IDs to delete.
```

**Estructura de los datos**:
```python
'records[]=id&records[]=id&records[]=id&records[]=id&records[]=id&records[]=id&records[]=id&records[]=id&records[]=id&records[]=id'
```

In [ ]:
# Delete Records

# Endpoint
endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

print(endpoint)

In [ ]:
# url-encoded

params = "&".join([f"records[]={id_}" for id_ in records_id[:10]])
params

In [ ]:
response = requests.delete(url = endpoint, params = params, headers = headers)

print(f"response: {response.status_code}")

print(f"endpoint: {response.url}")

print("-"*120)

pprint(response.json(), sort_dicts = False)

print("-"*120)

### 06. Delete a Record

```html
endpoint: https://api.airtable.com/v0/{BASE_ID}/{TABLE_ID}/{RECORD_ID}
HTTP method: DELETE
```

In [ ]:
# Delete a Record

# Endpoint

RECORD_ID = records_id[-1]
endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}/{RECORD_ID}"

print(endpoint)

In [ ]:
response = requests.delete(url = endpoint, headers = headers)

print(f"response: {response.status_code}")

print(f"endpoint: {response.url}")

print("-"*120)

pprint(response.json(), sort_dicts = False)

print("-"*120)

In [ ]:
################################################################################################################################